In [9]:
%load_ext autoreload
%autoreload 2

In [186]:
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import Graph
import folium
from shapely import wkt

In [2]:
#endpoint_url = "http://64.225.134.139:9999/Strabon/Query"
#sparql = SPARQLWrapper(endpoint_url)

In [128]:
graph = Graph()

# specify path to datasets
path_to_gadm_aut = "/mnt/data/GADM/RDF/gadm_AUT_RDF_population.ttl"
path_to_gadm_deu = "/mnt/data/GADM/RDF/gadm_DEU_RDF_population.ttl"
path_to_cams_obs = "/mnt/data/CAMS/RDF/observations/2020/01/cams_aq_2020_01_co.ttl"
path_to_obs_props = "/mnt/data/AQ_observable_properties.ttl"
path_to_cams_ref_raster = "/mnt/data/CAMS/RDF/CAMS_reference_grid_gadm_connections.ttl"

graph.parse(path_to_gadm_aut, format='ttl')
graph.parse(path_to_gadm_deu, format='ttl')
graph.parse(path_to_cams_obs, format='ttl')
graph.parse(path_to_obs_props, format='ttl')
graph.parse(path_to_cams_ref_raster, format='ttl')

<Graph identifier=N10574ed2584c4d88a297076308dc8c98 (<class 'rdflib.graph.Graph'>)>

In [190]:
# read sparql query
path_to_sparql_query = "../../scripts/sparql_queries/get_nr_days_treshold_exceedance_location.ttl"
with open(path_to_sparql_query, 'r') as query_file:
    query_string = query_file.read()

In [191]:
data = []
for row in graph.query(query_string):
    #obs_result, obs_time, foi_ent = row
    #print(foi_ent, obs_time, obs_result)
    print(row)
    data.append(row)

(rdflib.term.Literal('439.39584', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('2020-01-09', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date')), rdflib.term.Literal('POLYGON ((14.25 48.15, 14.35 48.15, 14.35 48.25, 14.25 48.25, 14.25 48.15))', datatype=rdflib.term.URIRef('http://www.opengis.net/ont/geosparql#wktLiteral')))


In [189]:
# Create a folium map centered around Linz
m = folium.Map(location=[48.15, 14.25], zoom_start=10)

# Iterate through the data and add markers to the map
for obs_result, obs_time, wkt_literal in data:
    # Convert Literal objects to Python types
    obs_result = float(obs_result)
    obs_time = obs_time.value
    geometry = wkt.loads(wkt_literal)

    # Create a GeoJSON representation of the geometry
    geojson = geometry.__geo_interface__

    # Create a GeoJson object and add it to the map
    folium.GeoJson(
        geojson,
        style_function=lambda feature: {
            'fillColor': 'green',  # Change the fill color as needed
            'fillOpacity': 0.5,
            'color': 'black',  # Change the border color as needed
            'weight': 2,
        },
        popup=f"Observation Result: {obs_result}<br>Observation Time: {obs_time}"
    ).add_to(m)

# Save the map to an HTML file
m.save('/mnt/data/maps/map.html')

### Construct sparql queries for template questions

#### 1) Request all values for specific time and location

What was the [specific pollutant] concentration in [city/municipality] for [month in year]?

In [68]:
 # List values and timestamps of CO2 for location in Linz (coordinate given)
query = """
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#> 
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>
PREFIX gadm: <http://example.com/ontologies/gadm#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 

SELECT ?obs_result ?obs_time 
WHERE {
    ?obs a sosa:Observation ;
       sosa:hasFeatureOfInterest ?foi_ent ;
       sosa:resultTime ?obs_time ;
       sosa:hasSimpleResult ?obs_result ;
       sosa:observedProperty ?obs_prop_ent .
    ?obs_prop_ent a sosa:ObservableProperty ;
       rdfs:label 'PM2.5' .
    ?foi_ent geo:hasGeometry ?foi_geom_ent .
    ?foi_geom_ent geo:asWKT ?foi_geom .

    FILTER (geof:sfWithin('''<http://www.opengis.net/def/crs/OGC/1.3/CRS84> POINT (14.36343 48.37428)'''^^geo:wktLiteral, ?foi_geom))
} 
LIMIT 10
"""

In [ ]:
for i, row in enumerate(graph.query(query)):
    pass

In [69]:
# Set the SPARQL query
sparql.setQuery(query)

# Set the return format to JSON
sparql.setReturnFormat(JSON)

try:
   # Execute the query and get the results
   ret = sparql.queryAndConvert()
   for r in ret["results"]["bindings"]:
      print(r["obs_result"]["value"], r["obs_time"]["value"])
except Exception as e:
    print(e)

KeyboardInterrupt: 